# `scrapy` basics

See [the scrapy tutorial online here](https://docs.scrapy.org/en/latest/intro/tutorial.html)

In this lesson we are going to explore Scrapy (`scrapy`), a Python library for specifying and running web scraping tasks. Codes that perform the systematic retrieval of remote resources are often called spiders or crawlers. Early examples of crawlers were those that were used to populate the search indexes of search engines like Altavista or Google.

What `scrapy` provides that `bs4` does not is a principled way to describe a scraping task from beginning to end. `bs4` focuses on manipulating a HTML (or HTML-like) document at hand; `scrapy` combines the retrieval step (which we did manually last time with a library like `requests`) and the extract step (which we did with `bs4`) into one artifact. Our objective is to replicate - with `scrapy` the scraping process sketched out in [`bs4` Further Topics](https://eamonnbell-dur.github.io/webscraping-for-humanities/bs4-further-topics.html). This introduction very closely follows the [Scrapy tutorial](https://docs.scrapy.org/en/latest/intro/tutorial.html).

The first thing to do is to initalise a `scrapy` project. You can do this at the commandline or in the cell below (`!` in a notebook cell passes the command to the shell - `bash` or similar)

In [ ]:
!scrapy startproject basics

As the message suggests, this has created a new directory (`basics`) in the current working directory of the notebook. In order to find out where this folder is, click `File > Open...` just underneath the Jupyter logo.

Now, enter the `basics/basics/spiders` folder and create a new file using the `New` button near the top right of the screen. Pick `Text File`. By clicking on the title, rename the file to `quotes_spider.py` and copy and paste the following script into the file:

### Source for `quotes_spider.py` - version 1

---

```python

from pathlib import Path

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'https://quotes.toscrape.com/page/1/',
            'https://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'quotes-{page}.html'
        Path(filename).write_bytes(response.body)
        self.log(f'Saved file {filename}')
```

In this script, notwithstanding some of the Python details, we can see there are two key tasks described: `start_requests`, which prepares a `scrapy.Request` for each URI in the list `urls`. The `callback=` argument suggests that the function `parse()` is called once for each of these requests - we might guess this happens after the HTTP request to the URI has been fired, and response has been recieved. Then - for each HTTP request (response) - four things happen, in order:

1. A bit of string processing puts the current page into a variable `page`.
2. We construct a filename using this variable.
3. We write the body of the HTTP response to a file on disk with this name.
4. We announce this fact to the world, via the `self.log()` function.

At this point, we could have done this with the Python standard library, or `requests`, or some combination of both of these. The smart thing about Scrapy is the way it passes information from the response to a request into functions for later processing (or, as we will later see, for firing off further requests).

Notice that we've given the spider a name: `"quotes"`. Because of the structure of the file and the directory tree that we created when we created the Scrapy project, we have a convenient way of running this spider, and we get nice logging for free. This is unlike when we work with `requests` alone. 

The command is `scrapy crawl [[spider_name]]`. (Note we have to `cd` into the project directory before we kick anything off).

In [ ]:
!cd basics; scrapy crawl quotes

If we parse the logs, we can see the message `Saved file quotes-1.html`. Similarly, we can double check that these files have been downloaded. (`head -n 20` shows the first ten lines of a file).

In [ ]:
!cd basics; head -n 20 quotes-1.html

In order to better understand the power of Scrapy, we are going to modify  `quotes_spider.py` just a little bit. This isn't something we'd do normally, but it serves to illustrate a point. Instead of writing the response body to a file, let's bundle a snippet of it (the first 10 characters) up with a little bit of metadata - in this case, the URI for the resource, the filename that the resource would have had. Scrapy uses the `yield` keyword to achieve this. 

### Source for `quotes_spider.py` - version 2

---


```python

from pathlib import Path

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'https://quotes.toscrape.com/page/1/',
            'https://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'quotes-{page}.html'
        
        yield {
            'filename': filename,
            'uri': response.url,
            'body_snippet': response.body[:10]
        }
```

In [ ]:
!cd basics; scrapy crawl quotes -O quotes.csv

In [ ]:
!cd basics; head quotes.csv

### Source for `quotes_spider.py` - version 3

---

```python
from pathlib import Path

import bs4
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'https://quotes.toscrape.com/page/1/',
            'https://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        soup = bs4.BeautifulSoup(response.text)
        
        yield {
            'uri': response.url,
            'title': soup.title.text,
        }
```

In [ ]:
!cd basics; scrapy crawl quotes -O quotes.csv

In [ ]:
!cd basics; head quotes.csv

At this point, you probably have enough to [go back to the final result with `bs4` from the last day's workshop](https://eamonnbell-dur.github.io/webscraping-for-humanities/bs4-further-topics.html#) and use what you know to create a new spider, called `discogs_spider.py`, which, given a URI of a Discogs.com list, will **yield** the album titles and the links to the cover images for each album in the list.

Think about how the script in `quotes_spider.py` could be adapted to work for Discogs.com before revealing the answer.

### Source for `discogs_spider.py` - version 1

---

```{toggle}

```python
from pathlib import Path

import bs4
import scrapy


class DiscogsSpider(scrapy.Spider):
    name = "discogs"

    def start_requests(self):
        urls = [
            'https://www.discogs.com/lists/277616',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        soup = bs4.BeautifulSoup(response.text)
        
        ol_albums = soup.find('ol', id='listitems')
        li_albums = ol_albums.find_all('li')

        for li in li_albums:
            album_title = li.find('a').get_text()
            cover_image_link = li.find('img')['src'] 
            yield {
                'uri': response.url,
                'album_title': album_title,
                'cover_image_link': cover_image_link
            }
```

```

Let's run the crawler...

In [ ]:
!cd basics; scrapy crawl discogs -O discogs.csv

...and look at the data it produces:

In [ ]:
!cd basics; head discogs.csv

We notice that the data returned relates only to the albums listed on the first page. We need to update the spider in such a way that it:

1. Identifies the link to the next page of results
2. Fires off a HTTP request to the URI that this link points to 
3. Parses the body of this request to extract the desired information
4. Go to step 1, stopping there if the link to the next page cannot be found (this means that we are on the last page of results)

Have a go at adapting the scraper for this task.

```{admonition} Hint
:class: tip
We know that the `yield` keyword is used inside `parse()` to expose the data that is transformed by the HTML parser so that it can be output to the user each time the spider is run. We also know that inside `start_requests()` the `yield` keyword can be used to trigger a request. There is no constraint on the number of `yield` statements in a function body; Python will get to each of them in turn. 

Python generators are a out of scope, [but you can read a bit about how they work here](https://wiki.python.org/moin/Generators) if you are interested. If you already know about them, [read the original proposal (PEP 255) that introduced them to the language](https://peps.python.org/pep-0255/).
```

### Source for `discogs_spider.py` - version 2

---

```{toggle}

```python
from pathlib import Path

import bs4
import scrapy


class DiscogsSpider(scrapy.Spider):
    name = "discogs"

    def start_requests(self):
        urls = [
            'https://www.discogs.com/lists/277616',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        soup = bs4.BeautifulSoup(response.text)
        
        ol_albums = soup.find('ol', id='listitems')
        li_albums = ol_albums.find_all('li')

        for li in li_albums:
            album_title = li.find('a').get_text()
            cover_image_link = li.find('img')['src'] 
            yield {
                'uri': response.url,
                'album_title': album_title,
                'cover_image_link': cover_image_link
            }

        nav = soup.find('nav', attrs={'aria-label':'Pagination'})
        link = nav.find('a', class_='pagination_next')
        link_destination = link.attrs['href']

        if link_destination is not None:
            absolute_next_link = 'https://www.discogs.com' + link_destination
            yield scrapy.Request(absolute_next_link, callback=self.parse)
```

```

(Don't forget to update your `basics/basics/spiders/discogs_scraper.py` file before proceeding!). Let's run the new scraper!

In [ ]:
!cd basics; scrapy crawl discogs -O discogs.csv

If we count the number of lines in the file, we'll see that over 150 results were retrieved. And, crucially, the scraper stopped correctly when it found that there was no next link on the page. Making sure that your spider has a well-defined stopping condition is key.

In [ ]:
!cd basics; wc -l discogs.csv

So we have arrived pretty much where we were at the end of the last tutorial, working instead with the Scrapy framework to describe a crawler. There are many advantages to this, but perhaps one of the most significant is the way that `scrapy` splits the responsibility for the scraping-related tasks into independent subsystems. Here is a picture from the Scrapy authors depicting its large-scale design:

![https://docs.scrapy.org/en/latest/_images/scrapy_architecture_02.png](https://docs.scrapy.org/en/latest/_images/scrapy_architecture_02.png)

One of the things that this allows for is the declarative specification of settings for the scraper, whether at the project level or at the level of the individual scraper. **Correctly setting these values is crucial for responsible scraping**. You can find the global settings inside the file `basics/basics/settings.py`. I exceprt some of the interesting settings below as they relate to:

- Setting the `USER_AGENT` string to identify yourself and your crawler
- Setting the crawler to respect `robots.txt`
- Setting up limits on the number of concurrent requests to remote resources (globally and on a per domain basis)
- Introduce a delay between requests (a basic form of ratelimiting/throttling) (see also: [the Scrapy documentation about Autothrottling](https://docs.scrapy.org/en/latest/topics/autothrottle.html?highlight=autothrottle))

I also include a setting about the encoding of the exported data, which may be relevant to you depending on the sites you are interested in studying.


```python
# Crawl responsibly by identifying yourself (and your website) on the user-agent
# Uncomment this line and fill in your own value here
#USER_AGENT = "basics (+http://www.yourdomain.com)"

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
CONCURRENT_REQUESTS = 32
# Configure maximum concurrent requests per domain performed by Scrapy 
CONCURRENT_REQUESTS_PER_DOMAIN = 16

# Configure a delay for requests for the same website (default: 0)
DOWNLOAD_DELAY = 3

# Set the encoding format for your item exports (this is a good choice but in case you
# are working with CJK sites this could need tweaking).
FEED_EXPORT_ENCODING = "utf-8"
```

In order to extend the scope of this scraper, an obvious focus would be the list of URIs provided in the spider to get the job running. Here we just use one URI, which acts as a "seed" for the spider, from which all the other requests "grow". But we may have several thousands of URIs to work with. As ever, there are many ways to move forward: these could either requested through repeated calls to the `scrapy.Request()` constructor, they could be provided as command-line arguments to the command that starts the crawl, or they could be retrieved from a global queue of URIs to process. These topics are out of scope for today, but I hope that you now have enough of an interest in Scrapy to continue exploring!